In [1]:
!rm -vrf '/content/tobit_loss'

removed '/content/tobit_loss/test/normal_cumulative_distribution_function_test.py'
removed '/content/tobit_loss/test/__init__.py'
removed directory '/content/tobit_loss/test'
removed '/content/tobit_loss/implementation/__init__.py'
removed '/content/tobit_loss/implementation/util.py'
removed '/content/tobit_loss/implementation/normal_cumulative_distribution_function.py'
removed '/content/tobit_loss/implementation/__pycache__/__init__.cpython-36.pyc'
removed '/content/tobit_loss/implementation/__pycache__/normal_cumulative_distribution_function.cpython-36.pyc'
removed '/content/tobit_loss/implementation/__pycache__/util.cpython-36.pyc'
removed directory '/content/tobit_loss/implementation/__pycache__'
removed '/content/tobit_loss/implementation/tobit_loss_module.py'
removed directory '/content/tobit_loss/implementation'
removed '/content/tobit_loss/tobit_loss.py'
removed '/content/tobit_loss/README.md'
removed '/content/tobit_loss/LICENSE'
removed '/content/tobit_loss/.gitignore'
remove

In [2]:
!git clone https://github.com/vlad-danaila/tobit_loss.git

Cloning into 'tobit_loss'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 8), reused 18 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [3]:
import sys
sys.path.insert(0, '/content/tobit_loss')
from implementation.normal_cumulative_distribution_function import cdf

In [4]:
from implementation.util import to_torch, to_numpy, normalize
import torch as t
from scipy.stats import norm
import unittest
import numpy as np
from implementation.normal_cumulative_distribution_function import cdf
from numpy.testing import assert_almost_equal

input = [10, 15, 20, 25, 30]

# manual gradient computing
x = np.array(input)
mean, std = x.mean(), x.std()
x_normalized = normalize(x, mean, std)
expected_cdf = norm.cdf(x_normalized)
expected_log_likelihood = np.log(expected_cdf)
expected_grad_log_likelihood_by_x = norm.pdf(x_normalized) / (expected_cdf * std)

# automatic gradient computing
x = to_torch(input, grad = True, device = 'cuda:0')
# in this test mean & std are considered constants
x_normalized = normalize(x, mean, std)
cdf_result = cdf(x_normalized)
assert_almost_equal(to_numpy(cdf_result), expected_cdf)

log_likelihood_result = t.log(cdf_result)
assert_almost_equal(to_numpy(log_likelihood_result), expected_log_likelihood)

loss = t.sum(log_likelihood_result)
loss.backward()

assert_almost_equal(to_numpy(x.grad), expected_grad_log_likelihood_by_x)